In [1]:
import pandas as pd

# Assembler/Assembly Language 
## Básicos de la programación

### Incluir librería

```
LIST P=xxfxx
#include <pxxfxx.inc>
```

### Configurar WORD (bit configuration)
- Ejemplo con pic 16f84

![WORD_CONFIG](src/WORD_CONFIG.png)

```
__config _CP_OFF&_PWRTE_ON&_WDTE_OFF&_FOSC_HS
```

- Ejemplo con pic 16f877

```
__config _CP_OFF&_DEBUG_OFF&_WRT_OFF&_CPD_OFF&_LVP_OFF&_BODEN_OFF&_PWRTE_ON&_WDTE_OFF&_FOSC_HS
```

### Entradas y salidas

TRIS son los registros para configurar los puertos como entradas ('1') o como salidas ('0').
PORT son los registros donde se guardan los valores de entrada o salida (según la configuración en TRIS). Ejemplo: "Si PORTA se configura como todo entrada en TRISA, y PORTB se configura como todo salida en TRISB, lo que se vea en el registro PORTA es lo que se recibe de afuera al PIC, y lo que se tiene en PORTB es lo que mostrará el PIC"

### TMR0

#### Temporizador
```
list p=16f84a
include <P16F84A.INC>

ORG 0x00
goto START

ORG 0x04
goto START

    ORG 0x23
START
    bsf STATUS, RP0 ; Pasar a banco 1
    movlw 0x00
    movwf TRISB ;Todo el puerto B como salida
    movlw 07h ;en binario 00000111 - máximo prescaler, asignado a TMR0; 
preescaler por 256 asignado a TMR0, cuenta pulsos CLK int (temporizador).
    movwf OPTION_REG ;this is the name of the register on the .INC of P16F84A
    bcf STATUS, RP0 ;Pasar a banco 0
    clrf TMR0 ;inicializa el timer0 a 0
MAIN
    movf TMR0,0 ;mueve el contenido de TMR0 a w
    andlw 0x01 ;máscara con el bit 0 de w ¿que pasa si andlw 0x02 y si andlw 0x80?
    movwf PORTB ;a puerto B
    goto main
end
```

***tpoCicloInstruccion[$\mu s$] * periodoRelojDeTrabajo * (256 - valorTMR0) * Preescaler***
; Donde TMR0max es _$2^8=256$_

Ejemplo: El tiempo de ciclo de instrucción (**CPI**) es 4, la frecuencia de trabajo es 4MHz, el preescaler asignado es 1:256, se tabulan los valores para diferentes máscaras calculando el periódo a la salida de PORTB

In [2]:
df = pd.DataFrame({
    'Máscara hexadecimal': ['0x01','0x02','0x04','0x08','0x10','0x20','0x40','0x80'],
    'Máscara decimal': ['1','2','4','8','16','32','64','128'],
    'Máscara binario': ['0000 0001','0000 0010','0000 0100','0000 1000','0001 0000','0010 0000','0100 0000','1000 0000'],
    'ValorTMR0': ['255','254','252','248','240','224','192','128'],
    'Bit PORTB': ['0','1','2','3','4','5','6','7'],
    'Periódo oscilación PORTB': ['$256\mu s$','$512\mu s$','$1024\mu s$','$2048\mu s$','$4096\mu s$','$8192\mu s$','$16.3840 ms$','$32.7680 ms$']})
df

,Máscara hexadecimal,Máscara decimal,Máscara binario,ValorTMR0,Bit PORTB,Periódo oscilación PORTB
0,0x01,1,0000 0001,255,0,$256\mu s$
1,0x02,2,0000 0010,254,1,$512\mu s$
2,0x04,4,0000 0100,252,2,$1024\mu s$
3,0x08,8,0000 1000,248,3,$2048\mu s$
4,0x10,16,0001 0000,240,4,$4096\mu s$
5,0x20,32,0010 0000,224,5,$8192\mu s$
6,0x40,64,0100 0000,192,6,$16.3840 ms$
7,0x80,128,1000 0000,128,7,$32.7680 ms$


#### Contador

```
list p=16f84a
include <P16F84A.INC>

ORG 0x00
goto START
ORG 0x04
goto START

ORG 0x23
START
    bsf STATUS, RP0 ; Pasar a banco 1
    movlw 0x00
    movwf TRISB ;Todo el puerto B como salida
    movlw 27h ;en binario 00100111 - máximo prescaler, asignado a TMR0; preescaler por 256 asignado a TMR0 por flanco de subida, cuenta pulsos de RA4
    movwf OPTION_REG ;this is the name of the register on the .INC of P16F84A
    bcf STATUS, RP0 ;Pasar a banco 0
    clrf TMR0 ;inicializa el timer0 a 0
MAIN
    movf TMR0,0 ;mueve el contenido de TMR0 a w
    andlw 0x01 ;máscara con el bit 0 de w ¿que pasa si andlw 0x02 y si andlw 0x80?
    movwf PORTB ;A puerto B
    goto main
end
```
***tpoCicloInstruccion[$\mu s$] * periodoRelojDeTrabajo * (TMR0max - valorTMR0) * Preescaler***
; Donde TMR0max es _$2^8=256$_

Ejemplo: El tiempo de ciclo de instrucción (**CPI**) es 4, la frecuencia de trabajo es 1KHz (***PERO SOLO AFECTAN LOS FLANCOS DE SUBIDA POR LO QUE EN REALIDAD LA FRECUENCIA ES 2kHz***), el preescaler asignado es 1:16, se tabulan los valores para diferentes máscaras calculando el periódo a la salida de PORTB

In [3]:
df = pd.DataFrame({
    'Máscara hexadecimal': ['0x01','0x02','0x04','0x08','0x10','0x20','0x40','0x80'],
    'Máscara decimal': ['1','2','4','8','16','32','64','128'],
    'Máscara binario': ['0000 0001','0000 0010','0000 0100','0000 1000','0001 0000','0010 0000','0100 0000','1000 0000'],
    'ValorTMR0': ['255','254','252','248','240','224','192','128'],
    'Bit PORTB': ['0','1','2','3','4','5','6','7'],
    'Periódo oscilación PORTB': ['$32 ms$','$64 ms$','$128 ms$','$256 ms$','$512 ms$','$1024 ms$','$2048 ms$','$4096 ms$']})
df

,Máscara hexadecimal,Máscara decimal,Máscara binario,ValorTMR0,Bit PORTB,Periódo oscilación PORTB
0,0x01,1,0000 0001,255,0,$32 ms$
1,0x02,2,0000 0010,254,1,$64 ms$
2,0x04,4,0000 0100,252,2,$128 ms$
3,0x08,8,0000 1000,248,3,$256 ms$
4,0x10,16,0001 0000,240,4,$512 ms$
5,0x20,32,0010 0000,224,5,$1024 ms$
6,0x40,64,0100 0000,192,6,$2048 ms$
7,0x80,128,1000 0000,128,7,$4096 ms$


### Tablas

- Uso general:
    - RETLW literal: returna W con un literal.
    - la cantidad 'n' de instrucciones máxima es 2^n°_bits; si el pic es de 8bits=> 256 instrucciones máximas

```
MAIN:
    movlw   0x02    ;load number 2 in W
    call    TABLE   ;calls table
                    ;W is now 0x0B
    goto    $
TABLE:
    addwf   PCL,f   ;adds W with PC and store result in PC
    retlw   0x0A    ;if W was 1
    retlw   0x0B    ;if W was 2 (which is this case)
    retlw   0x0C    ;if W was 3
    retlw   0x0D    ;if W was 4
    retlw   0x0E    ;if W was 5
    ...
    retlw   n       ;if W was n
```

- Ejemplo con 16f84: convierte de binario natural a código AIKEN-2421.
    - ***En este caso, la tabla se accesa sin ser llamada.***

```
LIST p=16f84
#include <p16f84.inc>

__config _CP_OFF&_PWRTE_ON&_WDTE_OFF&_FOSC_HS
;*********************************************************************
; Variables RAM
;*********************************************************************
Numero		EQU	0x30
Resultado	EQU	0x33
;*********************************************************************
; RESET VECTOR
;*********************************************************************
    org     0x00
    goto    MAIN
;*********************************************************************
; VECTOR INTERRUP
;*********************************************************************
	org 	0x0004
	goto	MAIN
;*********************************************************************

	org	10h
MAIN:
    call    AIKEN
    movwf   Resultado
    goto    MAIN
AIKEN:
    movfw   Numero
    sublw   b'00001001'
    btfss   STATUS,C	; minor than 10?
    goto    otros
menorQ10:
    movfw   Numero
    sublw   b'00000100'
    btfsc   STATUS,C	; greater than 4?
    goto    copiar
mayorQ4:
    movfw   Numero
    addwf   PCL,f
copiar:
    movfw   Numero
    RETURN
otros:
    movlw   0
    RETURN
    
    org	0x22
TABLA:
    retlw   0x0B
    retlw   0x0C
    retlw   0x0D
    retlw   0x0E
    retlw   0x0F
    
END
```

### Direccionamiento indirecto

```
; Ejercicio N° 7:
; Escriba un programa para inicializar en 55h un arreglo ubicado a partir de la dirección 43h.
; El tamaño del arreglo se encuentra en la dirección 2Ch.

; Resuúlvalo usando direccionamiento indirecto. 
; Validar revisando que el arreglo ha sido inicializado con MPLAB.
; ¿Es posible realizar el ejercicio usando exclusivamente direccionamiento directo? Fundamente.

LIST p=16f84
#include <p16f84.inc>

__config _CP_OFF&_PWRTE_ON&_WDTE_OFF&_FOSC_HS
;*********************************************************************
; Variables RAM
;*********************************************************************
tamanio	    EQU	    0x2C
inicio	    EQU	    0x43
;*********************************************************************
; RESET VECTOR
;*********************************************************************
    org     0x00
    goto    MAIN
;*********************************************************************
; VECTOR INTERRUP
;*********************************************************************
	org 	0x0004
	goto	MAIN
;*********************************************************************
	org	55h
MAIN:
    movfw   tamanio
    btfsc   STATUS,Z
    goto    $
    movlw   43h            ;cargo 43h en W
    movwf   FSR            ;lo guardo en FSR
    movlw   55h
    movwf   INDF
LOOP:
    decfsz  tamanio
    goto    incrementar
    goto    $
incrementar:
    incf    FSR
    movwf   INDF
    goto    LOOP
    
END
```

### Serial COM USART
##### Transmisor (Tx)

```
InitPortForTx:         		; In this case, we use PORTB
	BSF 	STATUS,RP0	;BANK 1
	movlw	b'11111111'
	movwf	TRISB		;port B as input
	BCF 	STATUS,RP0	;BANK 0
InitUsart:           		; In this case, PORTC contains Tx and Rx pins
	BANKSEL TRISC		;this is equivalent to "bsf status,RP0"
	BSF 	TRISC,6		;PIN Tx ; in this case
	BSF 	TRISC,7		;PIN Rx ; in this case
	BANKSEL PORTC		;this is equivalent to "bcf status, RP0"
	BSF     RCSTA,SPEN 		;Enable serial port
	BANKSEL PIE1	 	;BANK 1
	BCF 	PIE1,TXIE	;disable interruption by USART
	MOVLW	0x06		;USART async, Tx disable, 8 bits
	MOVWF	TXSTA		;BRGH set high speed,
	MOVLW	0X19		;baud rate of 9600 with X of 4 Mhz
	MOVWF 	SPBRG
	BCF 	STATUS,RP0	;BANK 0
Send:
	BSF 	STATUS,RP0	;BANK 1
	BSF 	TXSTA,TXEN	;enalbe tansmition
	BCF 	STATUS,RP0	;BANK 0
	MOVF	PORTB,0 	;load PORTB
	MOVWF	TXREG		;in transmition register
	BSF 	STATUS, RP0	;BANK 1
WaitSended:
	BTFSS	TXSTA,TRMT	;check if output shif tregister
	GOTO	WaitSended	;is empty
	BCF 	STATUS, RP0	;BANK 0
	GOTO	Send
    
	END
```

##### Receptor (Rx)

```
InitPortForRx:          	; In this case, we use PORTB
	BSF 	STATUS,RP0	;BANK 1
	movlw	b'00000000'
	movwf	TRISB		;port B as output
	BCF 	STATUS,RP0	;BANK 0
InitUsart:
	BANKSEL TRISC		;this is equivalent to "bsf status,RP0"
	BSF 	TRISC,6		;PIN Tx ; in this case
	BSF 	TRISC,7		;PIN Rx ; in this case
	BANKSEL PORTC		;this is equivalent to "bcf status, RP0"
	BSF     RCSTA,SPEN 		;Enable serial port
	BCF     RCSTA,RX9  		;8 bits para recepcion
	BANKSEL PIE1	 	;BANK 1
	BCF 	PIE1,RCIE	;disable interruption by USART
	MOVLW	0x06		;USART async, Tx disable, 8 bits
	MOVWF	TXSTA		;BRGH set high speed,
	MOVLW	0X19		;baud rate of 9600 with X of 4 Mhz
	MOVWF 	SPBRG
	BANKSEL RCSTA		;BANK 0
Receive:
	BSF 	RCSTA,CREN
WaitReceived:
	BTFSS	PIR1,RCIF	;check
	GOTO	WaitReceived	;if finished
	MOVF	RCREG,0
	MOVWF	PORTB
	GOTO	WaitReceived
	END
```